# Iterables

With our current setup we have graphs like this

![](http://nipy.org/nipype/_images/graphviz-b65596b443fdc105a6508acf176afcc6261aa65a.png)

Let's assume node B is smoothing, and we want to run the analysis with three different smoothing kernels. The resulting graph 
would look like this:
![](http://nipy.org/nipype/_images/graphviz-675dc232294d6cecf3284efeace35923746f8ac8.png)


This can be performed with [iterables](http://nipy.org/nipype/users/mapnode_and_iterables.html#iterables)

In [1]:
import os

home = os.path.expanduser('~')
data_path = os.path.join(home, 'nipype_data')
wd_path = os.path.join(data_path, '06_iterables')
if not os.path.exists(wd_path):
    os.mkdir(wd_path)
os.chdir(wd_path)
print(wd_path)

/Users/franzliem/nipype_data/06_iterables


# Iterate over multiple smoothing fwhm

In [2]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces import fsl
from nipype.interfaces.io import DataSink, SelectFiles

workflow_path = os.path.join(wd_path, 'wf')
wf = Workflow(name='smoothing_iterables_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

In [3]:
file_template = {'rs': 'in_data/s01/rs_10TRs.nii.gz'}
selectfiles = Node(SelectFiles(file_template, base_directory=data_path),
                   name="selectfiles")

In [4]:
moco = Node(fsl.MCFLIRT(), 'moco')

wf.connect(selectfiles, 'rs', moco, 'in_file')

Note the different definition of fwhm

In [5]:
smoothing = Node(fsl.IsotropicSmooth(), 'smoothing')
#smoothing.inputs.fwhm = 6
smoothing.iterables = ('fwhm', [4, 8, 12])
wf.connect(moco, 'out_file', smoothing, 'in_file')

In [6]:
ds_dir = os.path.join(wd_path, 'output_smoothing')
ds = Node(DataSink(), name='ds')
ds.inputs.base_directory = ds_dir

In [7]:
wf.connect(smoothing, 'out_file', ds, 'smoothed_data')

In [8]:
wf.run()

/Users/franzliem/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/franzliem/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node selectfiles in dir: /Users/franzliem/nipype_data/06_iterables/wf/smoothing_iterables_wf/selectfiles
INFO:workflow:Executing node moco in dir: /Users/franzliem/nipype_data/06_iterables/wf/smoothing_iterables_wf/moco
INFO:workflow:Running: mcflirt -in /Users/franzliem/nipype_data/in_data/s01/rs_10TRs.nii.gz -out /Users/franzliem/nipype_data/06_iterables/wf

# Iterate over multiple subjects

## At which step in the pipeline should we inject the different subjects?

In [9]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces import fsl
from nipype.interfaces.io import DataSink, SelectFiles

workflow_path = os.path.join(wd_path, 'wf')
wf = Workflow(name='subjects_iterables_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

Note that we now have an iterable to the input 'subject_id'.

This interable is a list of 2 strings ('s01', 's02').

In the rs string, '{subject_id}' is replaced by 's01' and 's02', respectively.

In [10]:
file_template = {'rs': 'in_data/{subject_id}/rs_10TRs.nii.gz'}
selectfiles = Node(SelectFiles(file_template, base_directory=data_path),
                   name="selectfiles")
selectfiles.iterables = ('subject_id', ['s01', 's02'])

In [11]:
moco = Node(fsl.MCFLIRT(), 'moco')

wf.connect(selectfiles, 'rs', moco, 'in_file')

In [12]:
smoothing = Node(fsl.IsotropicSmooth(), 'smoothing')
smoothing.inputs.fwhm = 6
wf.connect(moco, 'out_file', smoothing, 'in_file')

In [13]:
ds_dir = os.path.join(wd_path, 'output_subjects')
ds = Node(DataSink(), name='ds')
ds.inputs.base_directory = ds_dir
ds.inputs.regexp_substitutions = [
    ('_subject_id_', '')
]

In [14]:
wf.connect(smoothing, 'out_file', ds, 'smoothed_data')

In [15]:
wf.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node selectfiles.aI.a0 in dir: /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/selectfiles
INFO:workflow:Executing node moco.a0 in dir: /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/moco
INFO:workflow:Running: mcflirt -in /Users/franzliem/nipype_data/in_data/s01/rs_10TRs.nii.gz -out /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/moco/rs_10TRs_mcf.nii.gz
INFO:workflow:Executing node smoothing.a0 in dir: /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/smoothing
INFO:workflow:Running: fslmaths /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/moco/rs_10TRs_mcf.nii.gz -s 2.54797 /Users/franzliem/nipype_data/06_iterables/wf/subjects_iterables_wf/_subject_id_s01/smoothing/rs_10TRs_mcf_smooth.nii.gz
INFO:workflow:Exec

## Now run it on two cores

## Create your own pipeline!

# Debugging crash files

If your workflow crashes, take a look at the crash file directory. There should be a crash file that can be viewed with
the command line command:
    
__nipype_display_crash__